<a href="https://colab.research.google.com/github/Rob174/TIPE/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/TIPE'
import os

/content/drive/My Drive/TIPE


In [4]:
print("Vérification GPU")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print()
print()
from google.colab import files
import tensorflow as tf
import numpy as np
import scipy
from PIL import Image

# Ajoute images
entrainement = 60/100
test = 20/100
# validation = le reste : 20%
tailleImage = 399

images = []
noises = []

i = 1
while os.path.isfile("Images_source/clean/"+str(i)+".jpg"):
    images.append("Images_source/clean/"+str(i)+".jpg")
    i += 1

i = 1
while os.path.isfile("Images_source/noise/"+str(i)+".jpg"):
    noises.append("Images_source/noise/"+str(i)+".jpg")
    i += 1

Vérification GPU
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 708898455517088231
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5862816696497971332
physical_device_desc: "device: XLA_CPU device"
]




In [0]:
import cv2
import numpy as np
def next_batch(batch_size, noises, images,tailleAttendue):
    '''
    Return a total of `num` random samples and labels. 
    '''
    print("Size of {}".format(len(noises)))
    noiseTensor = []
    imageTensor = []
#     import matplotlib.pyplot as plt
    for i in range(batch_size):
      choix = np.random.randint(0,len(images))#choix aléatoire de l'image
      image = cv2.imread(images[choix],1)#Ouvre en rgb l'image nettoyée
      noise = cv2.imread(noises[choix],1)#Ouvre en rgb l'image bruitée
      angle = np.random.randint(0,90)
      taille = np.random.randint(int(tailleAttendue*2**0.5), image.shape[0]+1)# Racine de deux pour pouvoir toujours récupérer la taille désirée

      resizedImage = cv2.resize(image,(taille,taille))
      resizedNoise = cv2.resize(noise,(taille,taille))
      
      rows,cols = resizedImage.shape[:2]
#       print('CentreRow : ')
#       print((taille//2,rows-taille//2))
#       print('CentreCols : ')
#       print((taille//2,cols-taille//2))
#       tmp = taille//2
      
      centre = (np.random.randint(taille//2,rows-taille//2) if taille//2<rows-taille//2 else taille//2,np.random.randint(taille//2,cols-taille//2) if taille//2<cols-taille//2 else taille//2)
      M = cv2.getRotationMatrix2D(centre,angle,1)

      rotatedImage = cv2.warpAffine(resizedImage,M,(cols,rows))
      rotatedNoise = cv2.warpAffine(resizedNoise,M,(cols,rows))
#       print('Rotation : '+str(rotatedImage.shape))

      resultImage = rotatedImage[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
      resultNoise = rotatedNoise[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
#       print('Crop : '+str(resultImage.shape))
#       plt.imshow(resultImage)

      noiseTensor.append(np.array(resultNoise,np.uint8))
      imageTensor.append(np.array(resultImage,np.uint8))
#       print(imageTensor)
#     plt.show()
    return np.array(noiseTensor), np.array(imageTensor)
# next_batch(3,noises,images,200)

In [0]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 5
print("Essai avec "+str(i)+" couches")
nom = 'EssaiLotRemaniePlusNeurones'+str(i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    def denseLayer(input,nbNeurones,nom):
      dense = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(dense.get_shape()))
      return dense,tf.get_collection(tf.GraphKeys.VARIABLES, nom+'/kernel')[0],tf.get_collection(tf.GraphKeys.VARIABLES, nom+'/bias')[0]
    def generator(X,nbCouche,neurones):
        kernels = []
        bias = []
        i = 0
        couche,kernel,bias = denseLayer(X,3,'denseGenerator'+str(i))
        dense0 = tf.layers.dense(x,3,name='denseGenerator0')
        print('Shape of output : '+str(output.get_shape()))
        return output

    def discriminator(x):
        dense0 = tf.layers.dense(x,3,name='denseDiscriminator0')
        print('Shape of dense0 : '+str(dense0.get_shape()))
        dense1 = tf.layers.dense(dense0,3,name='denseDiscriminator1')
        print('Shape of dense1 : '+str(dense1.get_shape()))
        dense2 = tf.layers.dense(dense1,100,name='denseDiscriminator2')
        print('Shape of dense2 : '+str(dense2.get_shape()))
        dense3 = tf.layers.dense(dense2,100,name='denseDiscriminator3')
        print('Shape of dense3 : '+str(dense3.get_shape()))
        output = tf.layers.dense(dense3,3,name='denseDiscriminator4')
        print('Shape of output : '+str(output.get_shape()))
        return out_layer

    generator = generator(X)
    disc_vrai = discriminator()
    disc_faux = discriminator()
    
    gen_loss = -tf.reduce_mean(tf.log(disc_fake))
    disc_loss = -tf.reduce_mean(tf.log(disc_vrai) + tf.log(1. - disc_faux))
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    weights_gen = []
    bias_gen = []
    weights_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator0/kernel')[0])
    bias_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator0/bias')[0])
    weights_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator1/kernel')[0])
    bias_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator1/bias')[0])
    weights_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator2/kernel')[0])
    bias_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator2/bias')[0])
    weights_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator3/kernel')[0])
    bias_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator3/bias')[0])
    weights_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator4/kernel')[0])
    bias_gen.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseGenerator4/bias')[0])
    
    weights_disc = []
    bias_disc = []
    weights_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator0/kernel')[0])
    bias_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator0/bias')[0])
    weights_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator1/kernel')[0])
    bias_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator1/bias')[0])
    weights_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator2/kernel')[0])
    bias_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator2/bias')[0])
    weights_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator3/kernel')[0])
    bias_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator3/bias')[0])
    weights_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator4/kernel')[0])
    bias_disc.append(tf.get_collection(tf.GraphKeys.VARIABLES, 'denseDiscriminator4/bias')[0])
    
    gen_vars = [weights_gen,bias_gen]
    disc_vars = [weights_disc,bias_disc]
    
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
      global Lloss
      if len(Lloss) < duree:
          Lloss.append(lastLoss)
          return False
      else:
          Lloss = Lloss[1:duree] + [lastLoss]
          L = Lloss[:]
          L.sort(reverse=True)
          if L[0] <= seuil:
            if decroissance == True:
              return True if L == Lloss else False
            else:
              return True
          return False

    import os
    with tf.Session() as sess:
        init.run()
        for i in range(1,301)
          X_batch, y_batch = next_batch(batch_size, noises,images,height)
          sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
          if i % 5 == 0:
            summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
            summary_writer0.add_summary(summary_str,i)

            summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
            summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
            summary_writer0.add_summary(summary_strKernel0,i)
            summary_writer0.add_summary(summary_strBias0,i)

            summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
            summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
            summary_writer0.add_summary(summary_strKernel1,i)
            summary_writer0.add_summary(summary_strBias1,i)

            summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
            summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
            summary_writer0.add_summary(summary_strKernel2,i)
            summary_writer0.add_summary(summary_strBias2,i)

            summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
            summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
            summary_writer0.add_summary(summary_strKernel3,i)
            summary_writer0.add_summary(summary_strBias3,i)

            summary_strKernel4 = kernel_saver4.eval(feed_dict={X: X_batch, y: y_batch})
            summary_strBias4 = bias_saver4.eval(feed_dict={X: X_batch, y: y_batch})
            summary_writer0.add_summary(summary_strKernel4,i)
            summary_writer0.add_summary(summary_strBias4,i)
                  
      loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
      print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
      if arret(loss_train0,100,6) == True:
          break
  summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
  summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
  saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()